In [4]:
import dash
from dash import dcc, html
import pandas as pd
from dash.dependencies import Input, Output
import plotly.express as px
from urllib.request import urlopen
import io

# Initialize the app
app = dash.Dash(__name__)

# URL of the SpaceX launch geo data
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv'

# Fetch the data from the URL
response = urlopen(URL)

# Convert the binary data into a Pandas DataFrame
spacex_csv_file = io.BytesIO(response.read())
spacex_df = pd.read_csv(spacex_csv_file)

# Get the list of unique launch sites from the DataFrame
launch_sites = spacex_df['Launch Site'].unique()

# Create the options list for the Dropdown
dropdown_options = [{'label': 'All Sites', 'value': 'ALL'}]  # Adding the default 'ALL' option
for site in launch_sites:
    dropdown_options.append({'label': site, 'value': site})

# App layout with Dropdown, Pie Chart, Range Slider, and Scatter Plot
app.layout = html.Div([
    # Dropdown component for selecting launch sites
    dcc.Dropdown(
        id='site-dropdown',
        options=dropdown_options,  # Options list for the dropdown
        value='ALL',  # Default value is 'ALL'
        placeholder="Select a Launch Site here",  # Placeholder text
        searchable=True  # Allow searching in the dropdown
    ),
    
    # Pie chart for success/failure counts
    dcc.Graph(id='success-pie-chart'),

    # RangeSlider component for selecting Payload range
    dcc.RangeSlider(
        id='payload-slider',
        min=0,
        max=10000,
        step=1000,
        marks={i: str(i) for i in range(0, 10001, 1000)},  # Marks at every 1000 Kg
        value=[0, 10000],  # Default range for payload (from 0 to 10000)
        tooltip={"placement": "bottom", "always_visible": True}  # Show tooltip on hover
    ),
    
    # Scatter plot for Payload vs Success Outcome (with Booster Version Category)
    dcc.Graph(id='success-payload-scatter-chart')
])

# Callback for Success vs Failure Pie Chart
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
)
def get_pie_chart(selected_site):
    # Filter the DataFrame based on the selected site
    if selected_site == 'ALL':
        # If 'ALL' is selected, use all rows to generate the pie chart for all sites
        pie_data = spacex_df['class'].value_counts().reset_index()
        pie_data.columns = ['Launch Outcome', 'Count']
    else:
        # If a specific site is selected, filter the data for that site
        site_data = spacex_df[spacex_df['Launch Site'] == selected_site]
        pie_data = site_data['class'].value_counts().reset_index()
        pie_data.columns = ['Launch Outcome', 'Count']
    
    # Create the pie chart figure using Plotly
    fig = px.pie(pie_data, names='Launch Outcome', values='Count', title=f'Launch Success vs Failure for {selected_site}' if selected_site != 'ALL' else 'Total Launch Success vs Failure')

    return fig

# Callback for Success-Payload Scatter Plot (correlation between payload and success/failure)
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
     Input(component_id='payload-slider', component_property='value')]
)
def update_payload_success_scatter(selected_site, selected_payload_range):
    # Filter data based on selected payload range
    min_payload, max_payload = selected_payload_range
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= min_payload) & (spacex_df['Payload Mass (kg)'] <= max_payload)]
    
    # If a specific site is selected, filter further
    if selected_site != 'ALL':
        filtered_df = filtered_df[filtered_df['Launch Site'] == selected_site]
    
    # Create a scatter plot to visualize the correlation between payload and success
    fig = px.scatter(
        filtered_df, 
        x='Payload Mass (kg)', 
        y='class',
        title=f"Payload vs Mission Outcome for {selected_site if selected_site != 'ALL' else 'All Sites'}",
        labels={"class": "Mission Outcome (0=Failure, 1=Success)", "Payload Mass (kg)": "Payload Mass (kg)"},
        color='Booster Version Category',  # Color points by booster version
        color_continuous_scale="Viridis"  # Color scale for the booster version
    )

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

